In [2]:
import requests
import json
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import base64
from email.mime.text import MIMEText
from datetime import datetime, timedelta

# Set the Aeris Weather API credentials
CLIENT_ID = "YOUR_CLIENT_ID"
CLIENT_SECRET = "YOUR_CLIENT_SECRET"

# Set the Gmail API credentials
TOKEN_PATH = "YOUR_TOKEN_PATH"
CREDENTIALS_PATH = "YOUR_CREDENTIALS_PATH"
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

# Set the recipients of the email alerts
RECIPIENTS = ["jpatrickwisnewski@gmail.com", "pwisnewski@clubcarwash.com"]

def get_weather_alerts():
    # Set the location for which to get weather alerts
    location = "New York,NY"

    # Get the current time and the time one hour from now
    now = datetime.utcnow()
    later = now + timedelta(hours=1)

    # Format the time strings for the Aeris Weather API request
    now_str = now.strftime("%Y-%m-%dT%H:%M:%S")
    later_str = later.strftime("%Y-%m-%dT%H:%M:%S")

    # Set the parameters for the Aeris Weather API request
    params = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "filter": f"from:{now_str},to:{later_str}",
        "plimit": "1",
        "p": location,
        "fields": "details.producer, details.type, details.code, details.name, details.severity, details.response",
    }

    # Send the request to the Aeris Weather API
    response = requests.get("https://api.aerisapi.com/alerts/search", params=params)

    # Parse the response JSON
    data = json.loads(response.content.decode("utf-8"))

    # Get the alert details
    if data["success"]:
        alert = data["response"][0]["details"]
        return f"{alert['severity']} {alert['name']}: {alert['response']}"
    else:
        return None

def create_message(sender, to, subject, message_text):
    # Create the message
    message = MIMEText(message_text)
    message['to'] = to
    message['subject'] = subject
    message['from'] = sender

    # Encode the message as base64
    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

    return {'raw': encoded_message}

def send_email(service, message):
    try:
        # Send the email
        message = (service.users().messages().send(userId="me", body=message).execute())
        print(F'Sent message to {message["to"]}: Message Id: {message["id"]}')
    except HttpError as error:
        print(F'An error occurred: {error}')
        message = None
    return message

def main():
    # Load the Gmail API credentials
    credentials = Credentials.from_authorized_user_file(CREDENTIALS_PATH, SCOPES)
    service = build('gmail', 'v1', credentials=credentials)

    # Get the weather alerts
    alert_text = get_weather_alerts()

    # If there is an alert, send an email to each recipient
    if alert_text:
        for recipient in RECIPIENTS:
            message = create_message("sender@example.com", recipient, "Severe weather alert", alert_text)
            send_email(service, message)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'googleapiclient'